# Импорт необходимых библиотек
Импортируйте необходимые библиотеки, включая PyTorch.

In [9]:
# Импорт необходимых библиотек
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ssl

Вот список импортированных библиотек и их назначение:

1. `torch`: Это основная библиотека PyTorch, которая предоставляет основные функции для работы с тензорами и градиентами, а также инструменты для создания и обучения нейронных сетей.

2. `torch.nn`: Этот модуль PyTorch содержит классы для создания слоев нейронных сетей. Он также предоставляет функции потерь и другие полезные утилиты.

3. `torch.optim`: Этот модуль PyTorch содержит классы для различных алгоритмов оптимизации, которые можно использовать для обучения нейронных сетей, таких как SGD, Adam и другие.

4. `torchvision`: Эта библиотека предоставляет доступ к популярным наборам данных, моделям и инструментам преобразования изображений для компьютерного зрения.

5. `torchvision.transforms`: Этот модуль содержит классы для различных преобразований изображений, которые можно использовать для предварительной обработки и аугментации данных.

6. `torch.nn.functional`: Этот модуль содержит функциональные версии различных операций, используемых в нейронных сетях, таких как функции активации, функции потерь и т.д.

7. `ssl`: Это модуль Python для работы с SSL и TLS. В контексте этого кода, он может быть использован для обхода проверки SSL при загрузке данных.

# Загрузка и предварительная обработка датасета cifar10
Загрузите и предварительно обработайте датасет cifar10, используя torchvision.

In [10]:
# Загрузка и предварительная обработка датасета cifar10

# Преобразования, которые будут применены к изображениям
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# фикс для SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Загрузка обучающего набора данных
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Загрузка тестового набора данных
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Классы в датасете cifar10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Вот пошаговое объяснение кода:

1. `transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])`: Это преобразование, которое будет применено к каждому изображению в наборе данных. 

   - `transforms.ToTensor()`: Это преобразует изображения в формат тензоров PyTorch, который можно использовать для обучения модели.
   
   - `transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))`: Это нормализует каждый тензор изображения, приводя все значения к диапазону от -1 до 1. Это обычно помогает улучшить процесс обучения.

2. `ssl._create_default_https_context = ssl._create_unverified_context`: Это фикс для SSL, который позволяет загружать данные без проверки SSL сертификата.

3. `trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)`: Это загружает обучающий набор данных CIFAR10, применяя к каждому изображению указанное преобразование.

4. `trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)`: Это создает загрузчик данных, который можно использовать для итерации по обучающему набору данных во время обучения модели.

5. `testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)`: Это загружает тестовый набор данных CIFAR10, применяя к каждому изображению указанное преобразование.

6. `testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)`: Это создает загрузчик данных, который можно использовать для итерации по тестовому набору данных при оценке модели.

7. `classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')`: Это кортеж, содержащий имена классов в наборе данных CIFAR10.

# Определение архитектуры сверточной нейронной сети
Определите архитектуру сверточной нейронной сети, используя PyTorch.

In [11]:
# Определение архитектуры сверточной нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Сверточный слой (3 входных канала, 6 выходных каналов, ядро свертки 5x5)
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Слой пулинга (2x2)
        self.pool = nn.MaxPool2d(2, 2)
        # Сверточный слой (6 входных каналов, 16 выходных каналов, ядро свертки 5x5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Полносвязный слой (16 * 5 * 5 входных нейронов, 120 выходных нейронов)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # Полносвязный слой (120 входных нейронов, 84 выходных нейрона)
        self.fc2 = nn.Linear(120, 84)
        # Полносвязный слой (84 входных нейрона, 10 выходных нейронов)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Применяем свертку и пулинг
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Преобразуем тензор в вектор
        x = x.view(-1, 16 * 5 * 5)
        # Применяем полносвязные слои
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Создаем экземпляр сети
net = Net()

Вот пошаговое объяснение кода:

1. `class Net(nn.Module)`: Определение нового класса `Net`, который наследуется от `nn.Module`. `nn.Module` - это базовый класс для всех нейронных сетей в PyTorch.

2. `def __init__(self)`: Это конструктор класса, который вызывается при создании нового объекта этого класса.

3. `super(Net, self).__init__()`: Это вызов конструктора базового класса `nn.Module`.

4. `self.conv1 = nn.Conv2d(3, 6, 5)`: Создание сверточного слоя с 3 входными каналами (предположительно, RGB-изображение), 6 выходными каналами и ядром свертки размером 5x5.

5. `self.pool = nn.MaxPool2d(2, 2)`: Создание слоя максимального пулинга с размером окна 2x2.

6. `self.conv2 = nn.Conv2d(6, 16, 5)`: Создание второго сверточного слоя с 6 входными каналами, 16 выходными каналами и ядром свертки размером 5x5.

7. `self.fc1 = nn.Linear(16 * 5 * 5, 120)`: Создание полносвязного слоя с 16*5*5 входами (это размерность выхода предыдущего сверточного слоя) и 120 выходами.

8. `self.fc2 = nn.Linear(120, 84)`: Создание второго полносвязного слоя с 120 входами и 84 выходами.

9. `self.fc3 = nn.Linear(84, 10)`: Создание третьего полносвязного слоя с 84 входами и 10 выходами.

10. `def forward(self, x)`: Это метод, который определяет, как будет происходить прямой проход по сети.

11. `x = self.pool(F.relu(self.conv1(x)))`: Применение первого сверточного слоя, функции активации ReLU и слоя пулинга к входным данным.

12. `x = self.pool(F.relu(self.conv2(x)))`: Применение второго сверточного слоя, функции активации ReLU и слоя пулинга к результату предыдущего шага.

13. `x = x.view(-1, 16 * 5 * 5)`: Изменение формы тензора, чтобы его можно было передать в полносвязный слой.

14. `x = F.relu(self.fc1(x))`, `x = F.relu(self.fc2(x))`, `x = self.fc3(x)`: Применение полносвязных слоев и функции активации ReLU.

15. `net = Net()`: Создание экземпляра нейронной сети.

# Определение функции потерь и оптимизатора
Определите функцию потерь и оптимизатор, используя PyTorch.

In [12]:
# Определение функции потерь и оптимизатора
# Используем кросс-энтропийную потерю
criterion = nn.CrossEntropyLoss()

# Используем стохастический градиентный спуск для оптимизации
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Вот пошаговое объяснение кода:

1. `criterion = nn.CrossEntropyLoss()`: Создание объекта функции потерь. В данном случае используется кросс-энтропийная потеря, которая часто используется в задачах классификации. Эта функция сравнивает предсказания модели (вероятности для каждого класса) с истинными метками и вычисляет потери.

2. `optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)`: Создание объекта оптимизатора. В данном случае используется стохастический градиентный спуск (SGD). Оптимизатор используется для обновления весов модели на основе градиентов, вычисленных во время обратного распространения ошибки. `net.parameters()` передает все параметры модели (веса и смещения) в оптимизатор. `lr=0.001` задает скорость обучения, которая определяет, насколько сильно обновляются веса на каждом шаге. `momentum=0.9` добавляет механизм импульса к SGD, который помогает ускорить оптимизацию и избежать застревания в локальных минимумах.

# Обучение модели
Обучите модель, используя обучающий набор данных.

In [13]:
# Обучение модели
for epoch in range(2):  # Проходим по датасету несколько раз

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Получаем входные данные; data - это список [inputs, labels]
        inputs, labels = data

        # Обнуляем градиенты параметров
        optimizer.zero_grad()

        # Вперед + назад + оптимизация
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Печатаем статистику
        running_loss += loss.item()
        if i % 2000 == 1999:    # Печатаем каждые 2000 мини-пакетов
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.197
[1,  4000] loss: 1.878
[1,  6000] loss: 1.696
[1,  8000] loss: 1.607
[1, 10000] loss: 1.541
[1, 12000] loss: 1.492
[2,  2000] loss: 1.432
[2,  4000] loss: 1.397
[2,  6000] loss: 1.379
[2,  8000] loss: 1.349
[2, 10000] loss: 1.322
[2, 12000] loss: 1.293
Finished Training


Вот пошаговое объяснение кода:

1. `for epoch in range(2)`: Этот цикл проходит по датасету два раза (2 "эпохи"). Эпоха - это один полный проход по всему датасету.

2. `running_loss = 0.0`: Здесь инициализируется переменная для отслеживания общих потерь во время обучения.

3. `for i, data in enumerate(trainloader, 0)`: Этот цикл проходит по всем данным в `trainloader`. `trainloader` - это итератор, который предоставляет данные для обучения в мини-пакетах.

4. `inputs, labels = data`: Здесь из каждого мини-пакета данных извлекаются входные данные (`inputs`) и соответствующие им метки (`labels`).

5. `optimizer.zero_grad()`: Этот метод обнуляет градиенты всех параметров модели. Это важно делать перед каждым вызовом `loss.backward()`, иначе градиенты будут накапливаться.

6. `outputs = net(inputs)`: Здесь происходит прямой проход: модель (`net`) применяется к входным данным (`inputs`), и получаются предсказания модели (`outputs`).

7. `loss = criterion(outputs, labels)`: Здесь вычисляются потери, сравнивая предсказания модели (`outputs`) с истинными метками (`labels`).

8. `loss.backward()`: Этот метод вычисляет градиенты потерь по всем параметрам модели (весам и смещениям).

9. `optimizer.step()`: Этот метод обновляет все параметры модели, используя вычисленные градиенты. Это шаг оптимизации.

10. `running_loss += loss.item()`: Здесь к общим потерям добавляются потери текущего мини-пакета.

11. `if i % 2000 == 1999`: Это условие выполняется каждые 2000 мини-пакетов.

12. `print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))`: Здесь печатается средняя потеря за последние 2000 мини-пакетов.

13. `running_loss = 0.0`: Здесь общие потери обнуляются, чтобы начать отслеживание для следующих 2000 мини-пакетов.

14. `print('Finished Training')`: Это сообщение выводится, когда обучение завершено.

# Тестирование модели
Протестируйте модель на тестовом наборе данных и оцените ее производительность.

In [14]:
# Импорт необходимых библиотек
import torch

# Переводим модель в режим оценки
net.eval()

# Инициализируем счетчики правильных и общего числа предсказаний
correct = 0
total = 0

# Отключаем вычисление градиентов, так как они не нужны в процессе тестирования
with torch.no_grad():
    for data in testloader:
        # Получаем изображения и метки
        images, labels = data
        # Прогоняем изображения через сеть и получаем предсказания
        outputs = net(images)
        # Получаем предсказанные классы
        _, predicted = torch.max(outputs.data, 1)
        # Обновляем счетчики
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Выводим процент правильных предсказаний
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 53 %


Вот пошаговое объяснение кода:

1. `import torch`: Импорт библиотеки PyTorch, которая используется для обучения и тестирования нейронных сетей.

2. `net.eval()`: Перевод модели в режим оценки (или тестирования). В этом режиме PyTorch отключает некоторые функции, такие как dropout и batch normalization, которые используются во время обучения.

3. `correct = 0` и `total = 0`: Инициализация счетчиков для отслеживания общего числа предсказаний и числа правильных предсказаний.

4. `with torch.no_grad()`: В этом блоке PyTorch не будет вычислять градиенты. Это ускоряет вычисления и снижает потребление памяти, поскольку градиенты не нужны во время тестирования.

5. `for data in testloader`: Цикл по всем данным в `testloader`. `testloader` - это итератор, который предоставляет тестовые данные в мини-пакетах.

6. `images, labels = data`: Из каждого мини-пакета данных извлекаются изображения и соответствующие им метки.

7. `outputs = net(images)`: Прямой проход: модель (`net`) применяется к изображениям, и получаются предсказания модели.

8. `_, predicted = torch.max(outputs.data, 1)`: Получение предсказанных классов. `torch.max` возвращает максимальное значение и индекс максимального значения по указанной оси (в данном случае, ось 1, что соответствует классам).

9. `total += labels.size(0)`: Обновление общего числа предсказаний.

10. `correct += (predicted == labels).sum().item()`: Обновление числа правильных предсказаний. `(predicted == labels)` возвращает тензор булевых значений, где `True` соответствует правильным предсказаниям. `.sum().item()` суммирует все `True` значения и возвращает их как обычное число.

11. `print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))`: Вывод процента правильных предсказаний (точности) на тестовых данных.